# MIR Project Phase 2 -- classification

In [293]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import WordNetLemmatizer
from collections import Counter
import csv
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

<div dir="rtl">ابتدا متن را آماده میکنیم که این کار مشابه فاز اول انجام شده است با این تفاوت که تمام flag ها را true گذاشته ایم.</div>

In [294]:
def prepare_text(address, delete_punctuation, delete_stopWords, stemming):
        documents = []
        totalDocs = ""
        with open(address, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                doc = {}
                doc['title'] = row['title']
                doc['description'] = row['description']
                doc['views'] = row['views']
                totalDocs += row['description'].lower()+" "+row['title'].lower()
                documents.append(doc)

        totalDocs = nltk.word_tokenize(totalDocs)
        totalDocs = [i for i in totalDocs
                        if i.isalpha()]
        word = Counter(totalDocs)
        stopwords = [i[0] for i in word.most_common(37)]
        #print(stopwords)
        
        tokens = {}
        for i in range(len(documents)):
            doc = documents[i]
            title = doc.get('title').lower()
            title = nltk.word_tokenize(title)
            desc = doc.get('description').lower()
            desc = nltk.word_tokenize(desc)

            #### deleting punctuations
            if delete_punctuation=="T":
                title = [i
                for i in title
                    if i.isalpha()]
                desc = [i
                for i in desc
                    if i.isalpha()]

            ### finding and deleting stopwords
            if delete_stopWords=="T":
                title = [i
                for i in title
                    if i not in stopwords]
                if 'the' in title:
                    print('1')
                desc = [i
                for i in desc
                    if i not in stopwords]
            
            #### stemming
            if stemming=="T":
                stemmer = SnowballStemmer("english")
                title = [stemmer.stem(i)
                for i in title]
                desc = [stemmer.stem(i)
                for i in desc]
            
            #calculating the df of each word
            temp = set()
            for word in title:
                    temp.add(word)
            for word in desc:
                    temp.add(word)
            for word in temp:
                if word in tokens.keys():
                    tokens[word][0] += 1
                else:
                    tokens[word] = [1, len(tokens)]
            #changing the document to processed one
            documents[i]['title'] = title
            documents[i]['description'] = desc
       
        
        return documents, tokens

In [295]:
documents, tokens = prepare_text("data/train.csv", "T", "T", "T")
documents2, tokens2 = prepare_text("data/test.csv", "T", "T", "T")
print(len(tokens))
print(tokens)

10215
{'grow': [77, 0], 'quiet': [14, 1], 'simpl': [86, 2], 's': [78, 3], 'into': [233, 4], 'parkinson': [6, 5], 'face': [65, 6], 'four': [42, 7], 'morgan': [6, 8], 'older': [7, 9], 'found': [83, 10], 'distil': [3, 11], 'medit': [18, 12], 'loss': [21, 13], 'notic': [13, 14], 'when': [291, 15], 'age': [49, 16], 'herself': [23, 17], 'robin': [8, 18], 'diseas': [60, 19], 'experi': [114, 20], 'these': [131, 21], 'poem': [17, 22], 'power': [239, 23], 'poet': [17, 24], 'postrel': [1, 25], 'impress': [4, 26], 'defin': [16, 27], 'critic': [28, 28], 'virginia': [3, 29], 'which': [145, 30], 'cultur': [72, 31], 'design': [161, 32], 'true': [34, 33], 'imag': [65, 34], 'persuad': [6, 35], 'use': [309, 36], 'time': [199, 37], 'calcul': [5, 38], 'mean': [78, 39], 'glamour': [2, 40], 'polish': [2, 41], 'muse': [10, 42], 'care': [56, 43], 'ani': [35, 44], 'built': [38, 45], 'everyth': [40, 46], 'brain': [107, 47], 'explor': [109, 48], 'lie': [14, 49], 'role': [27, 50], 'sound': [45, 51], 'near': [26, 5

In [296]:
print(len(documents))
valDoc = documents[:230]
docdoc = documents[230:]

2295


<div dir="rtl">سپس label های داده های training و test را جدا کرده ایم:</div>

In [297]:
label = []
for i in range(len(documents)):
    for j in documents[i]:
        if j == "views":
            label += [documents[i][j]]
print(label)
vallabel = label[:230]
labellabel = label[230:]

label2 = []
for i in range(len(documents2)):
    for j in documents2[i]:
        if j == "views":
            label2 += [documents2[i][j]]
print(label2)

['-1', '-1', '1', '1', '1', '1', '1', '1', '-1', '-1', '1', '-1', '1', '-1', '1', '-1', '1', '1', '1', '1', '1', '1', '-1', '1', '1', '-1', '-1', '1', '1', '-1', '1', '-1', '-1', '1', '-1', '-1', '-1', '1', '1', '-1', '-1', '-1', '-1', '1', '1', '-1', '-1', '-1', '-1', '-1', '-1', '1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '1', '1', '-1', '-1', '-1', '1', '1', '-1', '-1', '1', '-1', '1', '1', '1', '-1', '-1', '1', '1', '1', '-1', '1', '1', '1', '-1', '-1', '-1', '-1', '-1', '1', '-1', '1', '1', '1', '-1', '-1', '-1', '-1', '-1', '1', '-1', '1', '-1', '1', '-1', '1', '-1', '1', '-1', '-1', '-1', '1', '-1', '-1', '1', '-1', '-1', '1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '1', '-1', '1', '1', '1', '1', '-1', '-1', '-1', '1', '-1', '1', '-1', '-1', '1', '-1', '-1', '-1', '1', '1', '1', '1', '-1', '1', '-1', '-1', '-1', '1', '-1', '1', '-1', '1', '1', '1', '-1', '-1', '-1', '1', '-1', '-1', '-1', '1', '-1', '1', '-1', '1', '1', '-1', '1', '1', '-1', '1', '-1

<div dir="rtl">به محاسبه ی idf می پردازیم:</div>

In [298]:
def idf_calculator(tokens, term, N):
    # print(term)
    if term in tokens:
        df = tokens[term][0]
        if(df != None):
              idf = math.log(N/df)
        else:
            return 0
        return idf
    else:
        return 0

<div dir="rtl">در نهایت به محاسبه ی tf idf پرداخته ایم:</div>

In [299]:
def tfidfwighter(tokens, DocID, documents):
    text = []
    vector = np.zeros((len(tokens),))
    text = documents[DocID].get('title') + documents[DocID].get('description')
    for word in text:
        tf = text.count(word)
        idf = idf_calculator(tokens, word , len(documents))
        if word in tokens:
            vector[tokens.get(word)[1]] = tf*idf
    return vector

<div dir="rtl">در این مرحله خصیصه ها را درون بردار x ریخته ایم:</div>

In [300]:
x = []
for i in range(len(documents)):
    x += [tfidfwighter(tokens, i, documents)]
# print(x)

xval = []
for i in range(len(valDoc)):
    xval += [tfidfwighter(tokens, i, valDoc)]

xdoc = []
for i in range(len(docdoc)):
    xdoc += [tfidfwighter(tokens, i, docdoc)]
print(xdoc)

[array([3.28908008, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 

<div dir="rtl">حال بردار های داده های تست را به دست می آوریم:</div>

In [301]:
x2 = []
for i in range(len(documents2)):
    x2 += [tfidfwighter(tokens, i, documents2)]


<div dir="rtl">تابع svm را به صورت زیر پیاده سازی کرده ایم:</div>

In [302]:
def svm(x , y ,c):
    svm_clf = SVC(C = c)
    svm_clf.fit(x,y)
    return svm_clf

<div dir="rtl">
    در ابن بخش ارزیابی را با استفاده از معیار های مختلف  روی دسته بند مشاهده میکنیم
</div>

In [303]:
def evaluater(test_docs, predicted_labels, name_of_classifier):
    tn, tp,fn,fp = 0,0,0,0
    for i in range(len(predicted_labels)):
        #positive
        if test_docs[i].get('views') == '1':
            #true positive
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tp += 1
            else:
                fp += 1
        #negative
        else:
            #true negative
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tn += 1
            else:
                fn += 1         
    precision = tp / (tp + fp)
    if tp+fn !=0:
        recall = tp / (tp + fn)
    else:
        recall =0
    accuracy = (tp + tn) / (tp+tn+fn+fp)
    if precision+ recall!=0:
        f1_score = 2 * precision * recall / (precision + recall)
    else:
        f1_score = 0
    print("the evaluation of " + name_of_classifier + " for class 1:" )
    print("recall = " + str(recall) )
    print("precision = "+ str(precision) )
    print("accuracy = " + str(accuracy) )
    print("f1_score = " + str(f1_score) )

In [304]:
def evaluater2(test_docs, predicted_labels, name_of_classifier):
    tn, tp,fn,fp = 0,0,0,0
    for i in range(len(predicted_labels)):
        #positive
        if test_docs[i].get('views') == '-1':
            #true positive
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tp += 1
            else:
                fp += 1
        #negative
        else:
            #true negative
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tn += 1
            else:
                fn += 1         
    precision = tp / (tp + fp)
    if tp+fn !=0:
        recall = tp / (tp + fn)
    else:
        recall =0
    accuracy = (tp + tn) / (tp+tn+fn+fp)
    if precision+ recall!=0:
        f1_score = 2 * precision * recall / (precision + recall)
    else:
        f1_score = 0
    print("the evaluation of " + name_of_classifier + " for class -1:" )
    print("recall = " + str(recall) )
    print("precision = "+ str(precision) )
    print("accuracy = " + str(accuracy) )
    print("f1_score = " + str(f1_score) )

<div dir="rtl">حال svm را به ترتیب برای c های ۱/۲ و ۱ و ۳/۲ و ۲ به دست می آوریم:توجه کنید که این داده ها بر روی داده های validation تست شده اند.</div>

In [306]:
print("Please enter c:")
c = float(input())
svm_clf=svm(xdoc,labellabel,c)
predict = svm_clf.predict(xval)

Please enter c:
0.5


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [307]:
evaluater(valDoc, predict, "SVM C=0.5")

the evaluation of SVM C=0.5 for class 1:
recall = 0.45652173913043476
precision = 1.0
accuracy = 0.45652173913043476
f1_score = 0.626865671641791


In [308]:
evaluater2(valDoc, predict, "SVM C=0.5")

the evaluation of SVM C=0.5 for class -1:
recall = 0
precision = 0.0
accuracy = 0.45652173913043476
f1_score = 0


<div dir="rtl">c=1:</div>

In [310]:
print("Please enter c:")
c = float(input())
svm_clf=svm(xdoc,labellabel,c)
predict = svm_clf.predict(xval)

Please enter c:
1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [311]:
evaluater(valDoc, predict, "SVM C=1")

the evaluation of SVM C=1 for class 1:
recall = 0.6050420168067226
precision = 0.6857142857142857
accuracy = 0.6521739130434783
f1_score = 0.6428571428571429


In [312]:
evaluater2(valDoc, predict, "SVM C=1")

the evaluation of SVM C=1 for class -1:
recall = 0.7027027027027027
precision = 0.624
accuracy = 0.6521739130434783
f1_score = 0.6610169491525424


<div dir="rtl">c=1.5:</div>

In [313]:
print("Please enter c:")
c = float(input())
svm_clf=svm(xdoc,labellabel,c)
predict = svm_clf.predict(xval)

Please enter c:
1.5


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [314]:
evaluater(valDoc, predict, "SVM C=1.5")

the evaluation of SVM C=1.5 for class 1:
recall = 0
precision = 0.0
accuracy = 0.5434782608695652
f1_score = 0


In [315]:
evaluater2(valDoc, predict, "SVM C=1.5")

the evaluation of SVM C=1.5 for class -1:
recall = 0.5434782608695652
precision = 1.0
accuracy = 0.5434782608695652
f1_score = 0.704225352112676


<div dir="rtl">c=2:</div>

In [316]:
print("Please enter c:")
c = float(input())
svm_clf=svm(xdoc,labellabel,c)
predict = svm_clf.predict(xval)
temp = 0
for i in range(len(valDoc)):
  if predict[i]!=valDoc[i]["views"]:
    temp+=1
print(100-((temp/len(valDoc))*100))

Please enter c:
2


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


54.34782608695652


In [317]:
evaluater(valDoc, predict, "SVM C=2")

the evaluation of SVM C=2 for class 1:
recall = 0
precision = 0.0
accuracy = 0.5434782608695652
f1_score = 0


In [318]:
evaluater2(valDoc, predict, "SVM C=2")

the evaluation of SVM C=2 for class -1:
recall = 0.5434782608695652
precision = 1.0
accuracy = 0.5434782608695652
f1_score = 0.704225352112676


<div dir="rtl">بهترین نتیجه روی داده های validation برای c=1 بود که در نتیجه نتایج این داده مهم است.</div>
<div dir="rtl">حال برای داده های تست این مقادیر را محاسبه میکنیم:</div>

In [319]:
c=1
svm_clf=svm(x,label,c)
predict = svm_clf.predict(x2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [320]:
evaluater(documents2, predict, "SVM C=1")

the evaluation of SVM C=1 for class 1:
recall = 0.0
precision = 0.0
accuracy = 0.5215686274509804
f1_score = 0


In [321]:
evaluater2(documents2, predict, "SVM C=1")

the evaluation of SVM C=1 for class -1:
recall = 0.5236220472440944
precision = 0.9925373134328358
accuracy = 0.5215686274509804
f1_score = 0.6855670103092784


<div dir="rtl">حال به بررسی random forest می پردازیم: با بررسی های مختلف به نتیجه ی زیر رسیدیم:</div>

In [342]:
rnd_clf = RandomForestClassifier(n_estimators=400, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [343]:
predict2 = rnd_clf.predict(x2)

In [344]:
evaluater(documents2, predict2, "Random forest")

the evaluation of Random forest for class 1:
recall = 0.5833333333333334
precision = 0.05785123966942149
accuracy = 0.5333333333333333
f1_score = 0.10526315789473684


In [345]:
evaluater2(documents2, predict, "Random forest")

the evaluation of Random forest for class -1:
recall = 0.5236220472440944
precision = 0.9925373134328358
accuracy = 0.5215686274509804
f1_score = 0.6855670103092784
